In [1]:
import pandas as pd
import json
import os

In [2]:
# список со всеми данными
dAll = []

# путь к файлам с json
pathJson = r"O:\Progs\Python\jsonToSav\data"


def getData(path: str):
    dJson = {}
    with open(path, "r", encoding="utf8") as f:
        dJson = json.load(f)
        dJson["filename"] = os.path.splitext(os.path.basename(path))[0]
        dJson1 = {k: v for k, v in dJson.items() if k != "products"}

        for d in dJson["products"]:
            dJson1.update(d)
            dAll.append(dJson1.copy())


# прохожусь по всем файлам и собираю json в единый словарь
with os.scandir(pathJson) as it:
    for entry in it:
        if entry.is_file(follow_symlinks=False) and entry.name.endswith(".json"):
            getData(entry.path)

# конвертирую в панду
df = pd.DataFrame(dAll)
df.head()

,date,shopAddress,shopName,totalSum,filename,category,name,quantity,price,sum
0,02.08.18 14:13,Большая Черемушкинская ул. 1 • Юго-Западный ад...,Лента,4620.0,t1,Дети,Д/п вода ФРУТОНЯНЯ детск пит артез 330мл,4.0,21.99,87.96
1,02.08.18 14:13,Большая Черемушкинская ул. 1 • Юго-Западный ад...,Лента,4620.0,t1,Дети,Ж/МЫЛО AQA BABY Д/МАЛЫША С ДОЗАТ 300МЛ,1.0,94.99,94.99
2,02.08.18 14:13,Большая Черемушкинская ул. 1 • Юго-Западный ад...,Лента,4620.0,t1,Дети,"Кефир АГУША детский 3,2% ТВА 204г",3.0,23.99,71.97
3,02.08.18 14:13,Большая Черемушкинская ул. 1 • Юго-Западный ад...,Лента,4620.0,t1,Дети,Д/п пюре ФРУТОНЯНЯ говяд с греч/мор100г,3.0,45.89,137.67
4,02.08.18 14:13,Большая Черемушкинская ул. 1 • Юго-Западный ад...,Лента,4620.0,t1,Дети,"Йогурт АГУША пит натурал 3,1% бут 200г",8.0,36.79,294.32


In [3]:
df.dtypes

date            object
shopAddress     object
shopName        object
totalSum       float64
filename        object
category        object
name            object
quantity       float64
price          float64
sum            float64
dtype: object

In [4]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # предположим, что если это не датафрейм, то серия
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    usage_kb = usage_b / 1024 # преобразуем байты в килобайты
    return "{:03.2f} KB".format(usage_kb)

In [5]:
mem_usage(df)

'26.11 KB'

In [6]:
# for col in ['quantity','price','sum']:
#    df[col] = pd.to_numeric(df[col])
df['quantity'] = df['quantity'].astype('int')
df.dtypes

date            object
shopAddress     object
shopName        object
totalSum       float64
filename        object
category        object
name            object
quantity         int32
price          float64
sum            float64
dtype: object

In [7]:
mem_usage(df)

'25.96 KB'

In [8]:
# выполняем понижающее преобразование 
# для столбцов типа int
df_int = df.select_dtypes(include=['int'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(df_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([df_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

0.27 KB
0.16 KB


,before,after
uint8,NaN,1.0
int32,1.0,NaN


In [9]:
# выполняем понижающее преобразование 
# для столбцов типа float
df_float = df.select_dtypes(include=['float'])
converted_float = df_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(df_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([df_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

0.99 KB
0.56 KB


,before,after
float32,NaN,3.0
float64,3.0,NaN


In [10]:
# создаем копию исходного датафрейма
optimized_df = df.copy()

# заменяем исходные числовые столбцы
# оптимизированными
optimized_df[converted_int.columns] = converted_int
optimized_df[converted_float.columns] = converted_float

# смотрим использование памяти
print(mem_usage(df))
print(mem_usage(optimized_df))
optimized_df.dtypes


25.96 KB
25.42 KB


date            object
shopAddress     object
shopName        object
totalSum       float32
filename        object
category        object
name            object
quantity         uint8
price          float32
sum            float32
dtype: object

In [11]:
optimized_df.describe()

,totalSum,quantity,price,sum
count,37.000000,37.000000,37.000000,37.000000
mean,4501.621094,1.783784,164.337845,131.351074
std,720.069153,2.029438,198.418518,112.597816
min,239.990005,0.000000,3.190000,9.570000
25%,4620.000000,1.000000,34.990002,61.990002
50%,4620.000000,1.000000,84.190002,89.089996
75%,4620.000000,1.000000,198.990005,178.990005
max,4620.000000,8.000000,774.989990,529.349976


In [12]:
# смотрим количество уникальных значений
# по каждому столбцу типа object
df_obj = optimized_df.select_dtypes(include=['object']).copy()
df_obj.describe()

,date,shopAddress,shopName,filename,category,name
count,37,37,37,37,37,37
unique,2,2,2,2,16,37
top,02.08.18 14:13,Большая Черемушкинская ул. 1 • Юго-Западный ад...,Лента,t1,Молочные продукты,Д/п пюре ФРУТОНЯНЯ говяд с греч/мор100г
freq,36,36,36,36,9,1


In [13]:
converted_obj = pd.DataFrame()

# пишем цикл, которой перебирает каждый столбец object, 
# проверяет его на соответствие заданному порогу 
# (количество уникальных значений должно быть меньше 50% 
# от общего количества значений), и если столбец 
# удовлетворяет порогу, преобразовывает его в тип category
for col in df_obj.columns:
    num_unique_values = len(df_obj[col].unique())
    num_total_values = len(df_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = df_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = df_obj[col]

In [14]:
# снова применяем функцию mem_usage, смотрим,
# сколько памяти занимают все столбцы типа object
# до и после преобразования в тип category
print(mem_usage(df_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([df_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

32.61 KB
11.65 KB


,before,after
object,6.0,1
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1


In [15]:
# смотрим, сколько памяти использует датафрейм
# после оптимизации типов
optimized_df[converted_obj.columns] = converted_obj
print(optimized_df.dtypes)
print(mem_usage(optimized_df))

date           category
shopAddress    category
shopName       category
totalSum        float32
filename       category
category       category
name             object
quantity          uint8
price           float32
sum             float32
dtype: object
12.12 KB


In [16]:
# смотрим, сколько памяти использует
# столбец date
date = optimized_df.date
print(mem_usage(date))
date.head()

0.38 KB


0    02.08.18 14:13
1    02.08.18 14:13
2    02.08.18 14:13
3    02.08.18 14:13
4    02.08.18 14:13
Name: date, dtype: category
Categories (2, object): [02.08.18 14:13, 14.01.20 17:39]

In [17]:
# преобразуем столбец 
# date в тип datetime 
optimized_df['date'] = pd.to_datetime(date,format='%d.%m.%y %H:%M')

# смотрим, сколько памяти использует
# столбец date
print(mem_usage(optimized_df))

12.15 KB


In [18]:
# создаем словарь, в котором ключами будут имена
# столбцов, а значениями - типы столбцов
dtypes = optimized_df.drop('date',axis=1).dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(column_types)

{   'category': 'category',
    'filename': 'category',
    'name': 'object',
    'price': 'float32',
    'quantity': 'uint8',
    'shopAddress': 'category',
    'shopName': 'category',
    'sum': 'float32',
    'totalSum': 'float32'}


In [19]:
print(mem_usage(df))
print(mem_usage(optimized_df))

33.62 KB
12.15 KB


In [20]:
optimized_df.dtypes

date           datetime64[ns]
shopAddress          category
shopName             category
totalSum              float32
filename             category
category             category
name                   object
quantity                uint8
price                 float32
sum                   float32
dtype: object

In [21]:
import pyreadstat

In [33]:
pyreadstat.write_sav(optimized_df.drop(["name","date"], axis=1), r"O:\Progs\Python\jsonToSav\data\spss1.sav")

ReadstatError: A provided string value was longer than the available storage size of the specified column

In [23]:
pyreadstat.write_sav(df, r"O:\Progs\Python\jsonToSav\data\spss1_df.sav")

In [41]:
import savReaderWriter as srw
# records = optimized_df.drop(["name","date"], axis=1).values.tolist()
records = optimized_df.values.tolist()

In [45]:
varTypes = {}
for col in optimized_df.columns:
    if str(optimized_df[col].dtype) == 'object':
        varTypes[col] = 1024
    elif 'date' in str(optimized_df[col].dtype):
        varTypes[col] = 0
    else:
        varTypes[col] = 0

varTypes
# del varTypes['name']
# del varTypes['date']
#varTypes

{'date': 0,
 'shopAddress': 0,
 'shopName': 0,
 'totalSum': 0,
 'filename': 0,
 'category': 0,
 'name': 1024,
 'quantity': 0,
 'price': 0,
 'sum': 0}

In [46]:
colsSave = list(optimized_df.columns)
colsSave
# colsSave.remove('name')
# colsSave.remove('date')
# colsSave

['date',
 'shopAddress',
 'shopName',
 'totalSum',
 'filename',
 'category',
 'name',
 'quantity',
 'price',
 'sum']

In [47]:
with srw.SavWriter(r"O:\Progs\Python\jsonToSav\data\spss.sav", colsSave, varTypes) as writer:
    for record in records:
        writer.writerow(record)

TypeError: unsupported operand type(s) for +=: 'error' and 'str'

In [48]:
vals = {}
for col in optimized_df.columns:
    if str(optimized_df[col].dtype) == 'category':
        optimized_df[col].cat.categories = [x[:30] for x in optimized_df[col].cat.categories]
        vals[col] = {i:x[:30] for i, x in enumerate(optimized_df[col].cat.categories)}
vals

{'shopAddress': {0: 'Большая Черемушкинская ул. 1 •',
  1: 'Новосибирская ул. 26 • Пермь'},
 'shopName': {0: 'Красное & Белое', 1: 'Лента'},
 'filename': {0: 't1', 1: 't2'},
 'category': {0: 'Алкоголь',
  1: 'Бакалея',
  2: 'Гастрономия',
  3: 'Дети',
  4: 'Для дома',
  5: 'Здоровье',
  6: 'Косметика',
  7: 'Кулинария',
  8: 'Молочные продукты',
  9: 'Мясо и птица',
  10: 'Напитки',
  11: 'Овощи и фрукты',
  12: 'Рыба',
  13: 'Снеки',
  14: 'Упаковка',
  15: 'Хлеб'}}

In [49]:
optimized_df.to_csv(r"O:\Progs\Python\jsonToSav\data\odf.txt", sep="\t", index=False)

In [51]:
s = "02.08.18 14:13"
s

'02.08.18 14:13'

In [54]:
b = s.encode()
b

b'02.08.18 14:13'

In [58]:
optimized_df['dateB'] = optimized_df['date'].astype('str')
optimized_df.head()

,date,shopAddress,shopName,totalSum,filename,category,name,quantity,price,sum,dateB
0,2018-08-02 14:13:00,Большая Черемушкинская ул. 1 •,Лента,4620.0,t1,Дети,Д/п вода ФРУТОНЯНЯ детск пит артез 330мл,4,21.990000,87.959999,2018-08-02 14:13:00
1,2018-08-02 14:13:00,Большая Черемушкинская ул. 1 •,Лента,4620.0,t1,Дети,Ж/МЫЛО AQA BABY Д/МАЛЫША С ДОЗАТ 300МЛ,1,94.989998,94.989998,2018-08-02 14:13:00
2,2018-08-02 14:13:00,Большая Черемушкинская ул. 1 •,Лента,4620.0,t1,Дети,"Кефир АГУША детский 3,2% ТВА 204г",3,23.990000,71.970001,2018-08-02 14:13:00
3,2018-08-02 14:13:00,Большая Черемушкинская ул. 1 •,Лента,4620.0,t1,Дети,Д/п пюре ФРУТОНЯНЯ говяд с греч/мор100г,3,45.889999,137.669998,2018-08-02 14:13:00
4,2018-08-02 14:13:00,Большая Черемушкинская ул. 1 •,Лента,4620.0,t1,Дети,"Йогурт АГУША пит натурал 3,1% бут 200г",8,36.790001,294.320007,2018-08-02 14:13:00


In [59]:
optimized_df.dtypes

date           datetime64[ns]
shopAddress          category
shopName             category
totalSum              float32
filename             category
category             category
name                   object
quantity                uint8
price                 float32
sum                   float32
dateB                  object
dtype: object